Этот код представляет собой пример использования машинного обучения и нейронных сетей для прогнозирования направления движения цен на финансовых рынках. Вот пошаговое объяснение:

1. **Загрузка данных: yfinance** используется для загрузки исторических данных цен на акции **Apple** (AAPL) с **Yahoo Finance**.
2. **Создание признаков: Next Close:** Добавление столбца со значениями цен закрытия следующего дня.
**Price Up:** Создание бинарного столбца, который равен **1**, если цена следующего дня ниже текущей, и **0** в противном случае.
3. **Подготовка данных:** Выбор признаков (**открытия**, **максимальной**, **минимальной**, **закрытия** и **объема**) и целевой переменной (**Price Up**).
Нормализация данных с использованием **MinMaxScaler**.
Разделение данных на обучающую и тестовую выборки.
4. **Создание нейронной сети:** Использование библиотеки **TensorFlow** и **Keras** для построения простой нейронной сети с тремя слоями (**64** нейрона, **32** нейрона, **1** нейрон) и функцией активации **relu** и **sigmoid**.
5. **Компиляция и обучение модели:** Компиляция модели с оптимизатором **adam** и функцией потерь **binary_crossentropy**. Обучение модели на обучающих данных в течение **50** эпох.
6. **Оценка точности:** Оценка точности модели на обучающей и тестовой выборках.
7. **Прогнозирование и визуализация:** Прогнозирование направления следующей цены на тестовых данных.
Создание столбца **Position** в данных для отметки моментов входа/выхода.
Визуализация с использованием библиотеки **Plotly:** линия тренда цены закрытия, фоновая заливка в зависимости от позиции, моменты входа/выхода, профиль тейка и стоп-лосс.
Вывод красиво оформленной таблицы с дополнительной информацией о каждой точке.
8. **Вывод результатов:** Вывод точности обучения и проверки.
Визуальное представление рекомендаций к позициям на графике и в таблице.

Этот код позволяет вам экспериментировать с прогнозированием направления цен на акции с использованием машинного обучения и предоставляет визуализацию результатов с моментами входа/выхода, профилем тейка и стоп-лоссом.

In [6]:
!pip install plotly prettytable

import numpy as np
import pandas as pd
import yfinance as yf
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from prettytable import PrettyTable
import plotly.graph_objects as go
import plotly.express as px

# Загрузка исторических данных по ценам акций
symbol = 'GC=F'
start_date = '2010-01-01'
end_date = '2023-11-27'
data = yf.download(symbol, start=start_date, end=end_date)

# Создание признаков
data['Next Close'] = data['Close'].shift(-1)

# Отбрасываем последнюю строку, так как у нее нет 'Next Close'
data = data.iloc[:-1, :]

# Создание признаков
data['Price Up'] = np.where(data['Close'] > data['Next Close'], 0, 1)

# Выбираем признаки
features = ['Open', 'High', 'Low', 'Close', 'Volume']
x = data[features].values
y = data['Price Up'].values

# Нормализация данных
scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(x)

# Разделение данных на обучающую и тестовую выборки
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=42)

# Модифицированная структура нейронной сети
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, input_dim=len(features), activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Компиляция модели
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Обучение модели
model.fit(x_train, y_train, epochs=200, batch_size=64, validation_data=(x_test, y_test))

# Оценка точности модели
accuracy_train = model.evaluate(x_train, y_train, verbose=0)[1] * 100
accuracy_test = model.evaluate(x_test, y_test, verbose=0)[1] * 100

print(f'Точность обучения: {accuracy_train:.2f}%')
print(f'Проверка точности: {accuracy_test:.2f}%')

# Прогнозирование направления следующей цены
prediction = model.predict(x_test)
predicted_labels = (prediction > 0.5).astype(int).flatten()

# Создание столбца с рекомендациями к данным
data['Position'] = 0  # Создаем столбец с нулевыми значениями
data.loc[data.index[-len(y_test):], 'Position'] = predicted_labels  # Заполняем значениями предсказаний

# Создание графика с моментами входа/выхода, профилем тейка и стоп-лоссом
fig = go.Figure()

# Линия тренда для цены закрытия
fig.add_trace(go.Scatter(x=data.index[-len(y_test):], y=data['Close'][-len(y_test):],
                         mode='lines', name='Цена закрытия', line=dict(color='black', width=2)))

# Заливка фона в зависимости от позиции
fig.add_trace(go.Scatter(x=data.index[-len(y_test):], y=[data['Close'].max()] * len(y_test),
                         mode='lines', name='Фон', fill='tonexty', fillcolor='rgba(0,100,80,0.2)', line=dict(color='rgba(255,255,255,0)')))

# Моменты входа/выхода, профиль тейка и стоп-лосс
for i in range(0, len(y_test), 10):  # Показываем каждую 10-ую точку для лучшей видимости
    if predicted_labels[i] == 1:  # Вход в сделку
        fig.add_trace(go.Scatter(x=[data.index[-len(y_test) + i]], y=[data['Close'].iloc[-len(y_test) + i]],
                                 mode='markers', marker=dict(color='blue', size=8), name='Вход в сделку'))
    if predicted_labels[i] == 1:  # Профиль тейка
        fig.add_trace(go.Scatter(x=[data.index[-len(y_test) + i]], y=[data['Close'].iloc[-len(y_test) + i] * 1.02],
                                 mode='markers', marker=dict(color='green', size=8), name='Профиль тейка'))
    if predicted_labels[i] == 1:  # Стоп-лосс
        fig.add_trace(go.Scatter(x=[data.index[-len(y_test) + i]], y=[data['Close'].iloc[-len(y_test) + i] * 0.98],
                                 mode='markers', marker=dict(color='red', size=8), name='Стоп-лосс'))

# Визуально красиво оформленная таблица
table = PrettyTable()
table.field_names = ['Дата', 'Цена закрытия', 'Рекомендация', 'Профиль тейка', 'Стоп-лосс']

for i in range(0, len(y_test), 10):
    table.add_row([data.index[-len(y_test) + i],
                   data['Close'].iloc[-len(y_test) + i],
                   'Вход в сделку' if predicted_labels[i] == 1 else 'Выход из сделки',
                   data['Close'].iloc[-len(y_test) + i] * 1.02 if predicted_labels[i] == 1 else '-',
                   data['Close'].iloc[-len(y_test) + i] * 0.98 if predicted_labels[i] == 1 else '-'])

print(table)

# Отображение графика
fig.update_layout(title='Движение цены с моментом входа, профилем тейка и стоп-лоссом',
                  xaxis_title='Дата',
                  yaxis_title='Цена',
                  legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
                  showlegend=True,
                  template='plotly_dark')  # Используем темный шаблон для профессионального вида
fig.show()



[*********************100%%**********************]  1 of 1 completed
Epoch 1/200


<ipython-input-6-995a40cbb389>:27: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



44/44 [==============================] - 2s 8ms/step - loss: 0.6938 - accuracy: 0.5131 - val_loss: 0.6930 - val_accuracy: 0.5293
Epoch 2/200
44/44 [==============================] - 0s 5ms/step - loss: 0.6943 - accuracy: 0.5063 - val_loss: 0.6922 - val_accuracy: 0.5293
Epoch 3/200
44/44 [==============================] - 0s 5ms/step - loss: 0.6940 - accuracy: 0.5052 - val_loss: 0.6931 - val_accuracy: 0.5293
Epoch 4/200
44/44 [==============================] - 0s 5ms/step - loss: 0.6935 - accuracy: 0.5156 - val_loss: 0.6913 - val_accuracy: 0.5293
Epoch 5/200
44/44 [==============================] - 0s 5ms/step - loss: 0.6927 - accuracy: 0.5234 - val_loss: 0.6924 - val_accuracy: 0.5293
Epoch 6/200
44/44 [==============================] - 0s 5ms/step - loss: 0.6931 - accuracy: 0.5131 - val_loss: 0.6921 - val_accuracy: 0.5293
Epoch 7/200
44/44 [==============================] - 0s 5ms/step - loss: 0.6921 - accuracy: 0.5195 - val_loss: 0.6921 - val_accuracy: 0.5308
Epoch 8/200
44/44 [======

<ipython-input-6-995a40cbb389>:72: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

